In [1]:
import pandas as pd
import numpy as np

%run function.ipynb

In [2]:
df = pd.read_csv('NTU_Drug.csv')
df_copy = df.copy()

c:\users\jack\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (3,4,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print(df_copy.shape)
df_copy.head(5)

(7570466, 11)


,datetime_UTC_8,deviceid,channel,invo_idx,id,name,quant,uniprice,totprice,invo_price,county_district
0,2021-02-14 16:40:00,17fd576a018ef61b1d78e9e7f6188fe479597faa,寶雅國際,68879107,1,媚點 零瑕美肌粧前乳(綠) 30g,1,220.0,220.0,736,臺中市大里區
1,2021-02-17 13:09:00,0e01fa942bec0321508e8c4bfb70261ba0d5c801,寶雅國際,74182246,6,折扣金額,1,-14.0,-14.0,124,臺中市大雅區
2,2021-02-27 21:58:00,358c116a685ba7a488820d54a7fd079234c89fc7,寶雅國際,45028186,6,do it now鋼繩伸縮吊環,1,60.0,60.0,1299,臺中市太平區
3,2021-02-18 17:36:00,871a78f697ddb680af42b5480f66cfdacef061f0,寶雅國際,37918202,6,思波綺瞬亮潤澤潤髮乳490ml,1,204.0,204.0,841,臺中市北 區
4,2021-02-21 20:15:00,7befa2b42131a70007d131731cecfbb4cb5cfd53,寶雅國際,39872805,6,折扣金額,1,-99.0,-99.0,306,臺中市北 區


In [4]:
na_check(df_copy)

,Missing Ratio
channel,0.001171
county_district,0.000478


In [5]:
df_copy = df_copy.dropna()
df_copy = df_copy[df_copy['name'] != '折扣金額']

In [6]:
df_copy.dtypes

datetime_UTC_8     object
deviceid           object
channel            object
invo_idx           object
id                 object
name               object
quant              object
uniprice           object
totprice           object
invo_price         object
county_district    object
dtype: object

In [7]:
df_copy = df_copy.drop(list(df_copy[df_copy['quant'] == 'quant'].index), axis=0)

In [8]:
df_copy[['uniprice', 'totprice', 'invo_price', 'quant']] = df_copy[['uniprice', 'totprice', 'invo_price', 'quant']].astype('float64')
df_copy['datetime_UTC_8'] = df_copy['datetime_UTC_8'].astype('datetime64')

In [9]:
df_copy = df_copy[df_copy['uniprice'] <= 10000]
df_copy = df_copy[df_copy['uniprice'] > 0]

In [10]:
# df_copy.to_csv('final_project_data.csv', index=False, encoding='utf-8', header=True)

In [14]:
channels_product = df_copy.groupby(by=['channel', 'name'], as_index=False).mean()
channels_product = channels_product[channels_product['uniprice'] <= 500]
channels_product.head(3)

,channel,name,quant,uniprice,totprice,invo_price
0,寶雅國際,!EP天然豬鬃毛澡刷PA-014,1.005208,79.0,79.411458,746.671875
1,寶雅國際,!EP普普印花澡巾PA-6643,1.010204,49.0,49.500000,779.591837
2,寶雅國際,!EP條紋澡巾PA-6011,1.000000,49.0,49.000000,768.883333


In [15]:
# channels_product.to_excel('通路商品單價.xlsx', index=False, encoding='utf-8')

In [18]:
invo_aggregate = df_copy.groupby(by=['channel', 'deviceid', 'invo_idx'], as_index=False).sum().drop(['uniprice', 'invo_price'], axis=1)
invo_aggregate.head(5)

,channel,deviceid,invo_idx,quant,totprice
0,寶雅國際,0000522bab667034d38fcdeaedd21f374bc69953,5020156,3.0,727.0
1,寶雅國際,0000522bab667034d38fcdeaedd21f374bc69953,25526549,6.0,535.0
2,寶雅國際,0000522bab667034d38fcdeaedd21f374bc69953,25526549,2.0,143.0
3,寶雅國際,000070dfd2e4130d6448b859cb4d36d438aa616d,48089638,3.0,658.0
4,寶雅國際,00009323d067515e0ecee7fb10902b08870dc67d,30107707,2.0,158.0


In [22]:
# 消費頻次
channels_freq = invo_aggregate.groupby(by=['channel', 'deviceid'], as_index=False).size()
channels_freq = channels_freq.groupby(by='channel', as_index=False).mean()
# channels_freq = channels_freq.rename(columns = {'invo_idx': 'avg_freq'})
channels_freq.head(5)

,channel,size
0,寶雅國際,2.110122
1,小三美日,1.279194
2,屈臣氏,1.963823
3,康是美,1.891966


In [46]:
# 單次消費品項數
channels_goods_num = df_copy.groupby(by=['channel', 'invo_idx'], as_index=False).size()
channels_goods_num = channels_goods_num.groupby(by='channel', as_index=False).mean()
channels_goods_num = channels_goods_num.rename(columns = {'size': 'avg_goods_num'})
channels_goods_num.head(5)

,channel,avg_goods_num
0,寶雅國際,4.224635
1,小三美日,2.261585
2,屈臣氏,2.524744
3,康是美,2.260013


In [47]:
# 單次消費總金額
channels_totprice = invo_aggregate.groupby(by='channel', as_index=False).median()
channels_totprice = channels_totprice.rename(columns = {'totprice': 'med_totprice'})
channels_totprice.head(5)

,channel,med_totprice
0,寶雅國際,297.00
1,小三美日,512.00
2,屈臣氏,302.64
3,康是美,327.00


In [48]:
channels_consumer_profile = channels_freq.merge(channels_goods_num, how='inner', on='channel').merge(channels_totprice, how='inner', on='channel')
channels_consumer_profile

,channel,avg_freq,avg_goods_num,med_totprice
0,寶雅國際,2.110122,4.224635,297.00
1,小三美日,1.279194,2.261585,512.00
2,屈臣氏,1.963823,2.524744,302.64
3,康是美,1.891966,2.260013,327.00


In [49]:
# channels_consumer_profile.to_excel('消費者輪廓.xlsx', index=False, encoding='utf-8')